# Bulk Delete EDMs (From Config File)

This notebook deletes EDMs (Exposure Data Models) from Moody's Risk Modeler based on database names defined in an Excel configuration file.

**Use Case:** Clean up EDMs that were created from a specific configuration file (e.g., test data cleanup).

**Process:**
1. Select an Excel configuration file from the `configuration/` folder
2. Check for existing analyses that use these EDMs (must be deleted first)
3. Read EDM names from the "Databases" sheet
4. Search for these EDMs in Moody's
5. Review found EDMs and any not found
6. Confirm deletion
7. Submit delete jobs and poll to completion
8. View summary of results

**Important:** Analyses must be deleted BEFORE their EDMs. If analyses exist, you will be prompted to use the Bulk Delete Analyses tool first.

**Warning:** This action permanently deletes EDMs from Moody's and CANNOT be undone!

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from pathlib import Path
from datetime import datetime
from helpers import ux
from helpers.irp_integration import IRPClient
from helpers.irp_integration.exceptions import IRPAPIError

# Initialize IRP client
irp_client = IRPClient()

ux.header("Bulk Delete EDMs (From Config File)")
ux.info("This tool deletes EDMs based on database names in an Excel configuration file.")

## 1. Select Configuration File

Place your Excel configuration file in the `configuration/` folder, then select it below.

In [ ]:
ux.subheader("Select Configuration File")

# Get the configuration directory (relative to this notebook)
config_dir = Path("configuration")
ux.info(f"Configuration directory: {config_dir.absolute()}")
print()

# Check if directory exists
if not config_dir.exists():
    ux.warning(f"Configuration directory does not exist: {config_dir}")
    ux.info("Creating directory...")
    config_dir.mkdir(parents=True, exist_ok=True)
    ux.success("Directory created")

# List available Excel files in the directory
excel_files = sorted(list(config_dir.glob("*.xlsx")) + list(config_dir.glob("*.xls")))

if not excel_files:
    ux.error("No Excel files found in configuration directory")
    print()
    ux.info(f"Please place your configuration file in: {config_dir.absolute()}")
    ux.info("The file should contain a 'Databases' sheet with a 'Database' column.")
    raise FileNotFoundError("No configuration files found in directory")

# Display available files
ux.info(f"Found {len(excel_files)} Excel file(s):")
print()
for i, f in enumerate(excel_files, 1):
    file_size = f.stat().st_size / 1024  # KB
    file_modified = datetime.fromtimestamp(f.stat().st_mtime).strftime('%Y-%m-%d %H:%M')
    ux.info(f"  [{i}] {f.name}")
    ux.info(f"      Size: {file_size:.2f} KB | Modified: {file_modified}")
    print()

# Get user selection
selection = ux.text_input(
    f"Select configuration file [1-{len(excel_files)}]",
    default="1"
)

if selection is None:
    ux.warning("Operation cancelled by user")
    raise Exception("User cancelled operation")

try:
    selected_index = int(selection) - 1
    if selected_index < 0 or selected_index >= len(excel_files):
        raise ValueError("Selection out of range")
    config_path = excel_files[selected_index]
except (ValueError, IndexError) as e:
    ux.error(f"Invalid selection: {selection}")
    raise ValueError(f"Invalid file selection. Please enter a number between 1 and {len(excel_files)}")

# Display selected file information
ux.success(f"Selected: {config_path.name}")
file_size = config_path.stat().st_size / 1024
file_modified = config_path.stat().st_mtime

file_info = [
    ["File Name", config_path.name],
    ["File Size", f"{file_size:.2f} KB"],
    ["Last Modified", datetime.fromtimestamp(file_modified).strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(file_info, headers=["Property", "Value"])

## 2. Check for Existing Analyses

Before deleting EDMs, we must ensure no analyses exist that use these EDMs. Analyses must be deleted first.

In [ ]:
ux.subheader("Checking for Existing Analyses")

# Try to read the Analysis Table sheet to check for analyses
analysis_check_required = False
existing_analyses = []

try:
    analysis_df = pd.read_excel(config_path, sheet_name='Analysis Table')
    
    # Check for required columns
    if 'Database' in analysis_df.columns and 'Analysis Name' in analysis_df.columns:
        # Extract unique EDM + Analysis Name combinations
        analysis_entries = analysis_df[['Database', 'Analysis Name']].dropna().drop_duplicates()
        
        if not analysis_entries.empty:
            ux.info(f"Found {len(analysis_entries)} analysis entry(ies) in config. Checking if they exist in Moody's...")
            print()
            
            # Check each analysis
            for _, row in analysis_entries.iterrows():
                edm_name = row['Database']
                analysis_name = row['Analysis Name']
                
                try:
                    filter_str = f'analysisName = "{analysis_name}" AND exposureName = "{edm_name}"'
                    results = irp_client.analysis.search_analyses(filter=filter_str)
                    
                    if results:
                        existing_analyses.extend(results)
                except IRPAPIError:
                    pass  # Analysis doesn't exist, which is fine
    else:
        ux.info("'Analysis Table' sheet found but missing required columns. Skipping analysis check.")
        
except ValueError:
    ux.info("No 'Analysis Table' sheet found in config. Skipping analysis check.")
except Exception as e:
    ux.warning(f"Could not read 'Analysis Table' sheet: {e}")
    ux.info("Proceeding without analysis check.")

# If analyses exist, stop execution
if existing_analyses:
    ux.error(f"BLOCKING: Found {len(existing_analyses)} existing analysis(es) in Moody's!")
    print()
    ux.warning("EDMs cannot be deleted while analyses still exist.")
    ux.warning("Deleting an EDM will fail if it has associated analyses.")
    print()
    
    # Show the existing analyses
    ux.subheader("Existing Analyses Found")
    analyses_data = []
    for a in existing_analyses:
        analyses_data.append({
            'EDM': a.get('exposureName', 'N/A'),
            'Analysis Name': a.get('analysisName'),
            'Analysis ID': a.get('analysisId'),
            'Peril': a.get('perilCode', 'N/A'),
            'Region': a.get('regionCode', 'N/A')
        })
    
    analyses_display_df = pd.DataFrame(analyses_data)
    ux.dataframe(analyses_display_df, title=f"Analyses That Must Be Deleted First ({len(analyses_display_df)} total)")
    
    print()
    ux.error("ACTION REQUIRED: Please run 'Bulk Delete Analyses (From Config).ipynb' first.")
    ux.info("After deleting the analyses, return to this notebook to delete the EDMs.")
    raise Exception("Cannot delete EDMs: Existing analyses must be deleted first")
else:
    ux.success("No existing analyses found. Safe to proceed with EDM deletion.")

## 3. Read EDM Names from Configuration

Reading the "Databases" tab and extracting EDM names from the `Database` column.

In [ ]:
ux.subheader("Reading Configuration File")

try:
    # Read the Databases sheet
    df = pd.read_excel(config_path, sheet_name='Databases')
    ux.success(f"Successfully read 'Databases' sheet")
    
except ValueError as e:
    if "Databases" in str(e):
        ux.error("'Databases' sheet not found in the configuration file")
        ux.info("The Excel file must contain a sheet named 'Databases'")
    raise
except Exception as e:
    ux.error(f"Failed to read configuration file: {e}")
    raise

# Check for Database column
if 'Database' not in df.columns:
    ux.error("'Database' column not found in the Databases sheet")
    ux.info(f"Available columns: {', '.join(df.columns)}")
    raise ValueError("Missing required 'Database' column")

# Extract unique EDM names
edm_names = df['Database'].dropna().unique().tolist()

if not edm_names:
    ux.warning("No EDM names found in the 'Database' column")
    raise ValueError("No EDMs to process")

ux.success(f"Found {len(edm_names)} unique EDM name(s) in configuration")
print()

# Display EDM names
ux.subheader("EDMs from Configuration")
edm_list_data = [[i, name] for i, name in enumerate(edm_names, 1)]
ux.table(edm_list_data, headers=["#", "EDM Name"])

## 4. Search for EDMs in Moody's

Searching for each EDM by exact name match in the Moody's API.

In [ ]:
ux.subheader("Searching Moody's API")
ux.info(f"Searching for {len(edm_names)} EDM(s)...")
print()

found_edms = []
not_found_edms = []

for i, edm_name in enumerate(edm_names, 1):
    try:
        # Search for exact EDM name match using search_edms with filter
        filter_str = f'exposureName = "{edm_name}"'
        results = irp_client.edm.search_edms(filter=filter_str)

        if results and len(results) > 0:
            found_edms.append(results[0])
            ux.success(f"  [{i}/{len(edm_names)}] Found: {edm_name}")
        else:
            not_found_edms.append(edm_name)
            ux.warning(f"  [{i}/{len(edm_names)}] Not found: {edm_name}")

    except IRPAPIError as e:
        not_found_edms.append(edm_name)
        ux.warning(f"  [{i}/{len(edm_names)}] Not found: {edm_name}")

print()
ux.info(f"Search complete: {len(found_edms)} found, {len(not_found_edms)} not found")

## 5. Review EDMs to Delete

Review the EDMs that were found and will be deleted.

In [ ]:
# Show EDMs not found (if any)
if not_found_edms:
    ux.subheader(f"EDMs Not Found ({len(not_found_edms)})")
    ux.warning("The following EDMs were not found in Moody's (already deleted or never created):")
    for name in not_found_edms:
        ux.info(f"  - {name}")
    print()

# Show found EDMs
if not found_edms:
    ux.warning("No EDMs found in Moody's to delete")
    ux.info("All EDMs from the configuration are either already deleted or were never created.")
    raise Exception("No EDMs to delete")

ux.subheader(f"EDMs Found ({len(found_edms)})")

# Build DataFrame for display
edms_data = []
for e in found_edms:
    edms_data.append({
        'Exposure ID': e.get('exposureId'),
        'EDM Name': e.get('exposureName'),
        'Num Portfolios': e.get('metrics').get('portfolioCount', 0),
        'Num Accounts': e.get('metrics').get('accountCount', 0),
        'Num Locations': e.get('metrics').get('locationCount', 0),
        'Created': e.get('createdAt', 'N/A')
    })

edms_df = pd.DataFrame(edms_data)
ux.dataframe(edms_df, title=f"EDMs to Delete ({len(edms_df)} total)")

# Store for deletion step
edms_to_delete = found_edms.copy()

print()
ux.warning(f"The above {len(edms_to_delete)} EDM(s) will be PERMANENTLY DELETED from Moody's.")

## 6. Confirm Deletion

**This action cannot be undone!** Please review the EDMs listed above carefully before confirming.

In [ ]:
ux.header("CONFIRMATION REQUIRED")
ux.error(f"You are about to PERMANENTLY DELETE {len(edms_to_delete)} EDM(s)")
ux.info(f"Configuration file: {config_path.name}")
print()

# First confirmation
if not ux.yes_no("Are you sure you want to delete these EDMs?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

# Second confirmation with count
if not ux.yes_no(f"Final confirmation: DELETE {len(edms_to_delete)} EDM(s)?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

ux.success("Confirmation received. Starting deletion...")
print()

## 7. Delete EDMs

Submitting delete jobs and polling to completion. This may take some time.

In [ ]:
# Track results
submitted_jobs = []  # List of (edm_name, exposure_id, job_id)
submission_errors = []

ux.subheader("Submitting Delete Jobs")
ux.info("Submitting all delete jobs first, then polling in batch...")
print()

for i, edm in enumerate(edms_to_delete, 1):
    edm_name = edm.get('exposureName', 'Unknown')
    exposure_id = edm.get('exposureId')

    try:
        # Submit delete job (does not poll - just submits)
        job_id = irp_client.edm.submit_delete_edm_job(exposure_id)
        submitted_jobs.append((edm_name, exposure_id, job_id))
        ux.success(f"  [{i}/{len(edms_to_delete)}] Submitted: {edm_name} (Job ID: {job_id})")

    except Exception as e:
        error_msg = f"{edm_name}: {str(e)}"
        submission_errors.append(error_msg)
        ux.error(f"  [{i}/{len(edms_to_delete)}] FAILED to submit: {edm_name}")
        ux.warning(f"      Error: {str(e)[:100]}")

print()
ux.info(f"Submitted {len(submitted_jobs)} job(s), {len(submission_errors)} submission error(s)")

In [ ]:
# Poll all jobs in batch
deleted_count = 0
failed_count = 0
errors = list(submission_errors)  # Start with submission errors
job_results = []

if submitted_jobs:
    ux.subheader("Polling Jobs to Completion")
    ux.info(f"Waiting for {len(submitted_jobs)} delete job(s) to complete...")
    print()

    # Extract job IDs for batch polling
    job_ids = [job_id for (_, _, job_id) in submitted_jobs]
    # Use int keys for consistent lookup (API may return jobId as int or str)
    edm_name_by_job_id = {int(job_id): edm_name for (edm_name, _, job_id) in submitted_jobs}

    try:
        # Poll all jobs in batch
        final_statuses = irp_client.job.poll_risk_data_job_batch_to_completion(job_ids)

        # Process results
        for job_status in final_statuses:
            # Convert jobId to int for consistent lookup
            job_id = int(job_status.get('jobId', 0))
            status = job_status.get('status', 'UNKNOWN')
            edm_name = edm_name_by_job_id.get(job_id, f'Unknown (Job {job_id})')

            if status == 'FINISHED':
                deleted_count += 1
                ux.success(f"  Deleted: {edm_name}")
                job_results.append({
                    'edm_name': edm_name,
                    'status': 'DELETED',
                    'job_status': status
                })
            else:
                failed_count += 1
                error_msg = f"{edm_name}: Job finished with status {status}"
                errors.append(error_msg)
                ux.error(f"  FAILED: {edm_name} (status: {status})")
                job_results.append({
                    'edm_name': edm_name,
                    'status': 'FAILED',
                    'job_status': status
                })

    except Exception as e:
        # If batch polling fails, mark all as failed
        ux.error(f"Batch polling failed: {str(e)[:100]}")
        for (edm_name, _, job_id) in submitted_jobs:
            failed_count += 1
            errors.append(f"{edm_name}: Polling failed - {str(e)}")
            job_results.append({
                'edm_name': edm_name,
                'status': 'ERROR',
                'job_status': 'Polling failed'
            })

    print()
else:
    ux.warning("No jobs were successfully submitted.")

## 8. Deletion Summary

In [ ]:
ux.header("Deletion Summary")

# Summary statistics
summary_data = [
    ["Configuration File", config_path.name],
    ["EDMs in Config", len(edm_names)],
    ["EDMs Found in Moody's", len(found_edms)],
    ["EDMs Not Found", len(not_found_edms)],
    ["Successfully Deleted", deleted_count],
    ["Failed to Delete", failed_count],
]

ux.table(summary_data, headers=["Metric", "Value"])
print()

# Overall status
if failed_count == 0 and deleted_count > 0:
    ux.success(f"All {deleted_count} EDM(s) deleted successfully!")
elif deleted_count == 0 and failed_count > 0:
    ux.error(f"All {failed_count} deletion(s) failed")
elif deleted_count > 0 and failed_count > 0:
    ux.warning(f"Partial success: {deleted_count} deleted, {failed_count} failed")

# Show job results
if job_results:
    print()
    ux.subheader("Job Results")
    job_results_df = pd.DataFrame(job_results)
    job_results_df.columns = ['EDM Name', 'Result', 'Job Status']
    ux.dataframe(job_results_df, title="Delete Job Results")

# Show EDMs not found
if not_found_edms:
    print()
    ux.subheader("EDMs Not Found (Skipped)")
    for name in not_found_edms:
        ux.info(f"  - {name}")

# Show errors if any
if errors:
    print()
    ux.subheader("Deletion Errors")
    for error in errors:
        ux.error(f"  - {error}")

print()
ux.info("Note: Deleted EDMs cannot be recovered. If you need to recreate them,")
ux.info("you will need to re-run the EDM creation workflow with the configuration file.")

---